In [1]:
from chronosx_train_eval import *

In [2]:
time_series, past_covs, future_covs = load_data(
    data_path="~/models/uni2ts/data/demand_data_all_cleaned_featsel.csv",
    target="actual",
)

Successfully loaded data from /home/lianlian/models/uni2ts/data/demand_data_all_cleaned_featsel.csv
Data shape: (61369, 8)
Columns: ['actual', 'met', 'period', 'hour', 'tempC', 'heatIndexC', 'feelsLikeC', 'humidity']


In [3]:
time_series

array([4939.7915, 4837.298 , 4751.866 , ..., 6248.124 , 6011.4326,
       5831.5137], dtype=float32)

In [4]:
past_covs

array([[4.8017e+03, 3.0000e+00, 0.0000e+00, ..., 2.9000e+01, 2.9000e+01,
        8.4000e+01],
       [4.7013e+03, 3.0000e+00, 1.0000e+00, ..., 2.9000e+01, 2.9000e+01,
        8.4000e+01],
       [4.6196e+03, 3.0000e+00, 2.0000e+00, ..., 2.9000e+01, 2.9000e+01,
        8.5000e+01],
       ...,
       [6.1200e+03, 1.0000e+00, 2.2000e+01, ..., 2.8000e+01, 2.8000e+01,
        7.6000e+01],
       [5.8851e+03, 1.0000e+00, 2.3000e+01, ..., 2.7000e+01, 2.7000e+01,
        7.9000e+01],
       [5.7059e+03, 3.0000e+00, 0.0000e+00, ..., 2.7000e+01, 2.7000e+01,
        8.1000e+01]], dtype=float32)

In [5]:
future_covs

In [6]:
model_config = {
    "base_model_path": "amazon/chronos-t5-tiny",  # or other supported models
    "model_type": "chronos",
    "covariate_dim": 5,
    "hidden_dim": 256,
    "use_past_covariates": True,
    "use_future_covariates": False,
    "freeze_pretrained": True,
}

training_config = {
    "learning_rate": 1e-3,
    "batch_size": 16,
    "num_epochs": 50,
    "context_length": 512,
    "prediction_length": 72,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "save_dir": "./experiment_results",
    
    "patience": 10,
    "num_eval_samples": 50,
}

data_config = {
    "data_path": '~/models/uni2ts/data/demand_data_all_cleaned_featsel.csv',
    "train_ratio": 0.6,  # 60% training
    "val_ratio": 0.2,   # 20% validation, 20% test (1 - 0.6 - 0.2 = 0.2)
}

In [7]:
results = run_full_experiment(model_config, training_config, data_config)

print("\nExperiment Results:")
print(f"Final Training Loss: {results['final_train_loss']:.6f}")
print(f"Final Validation Loss: {results['final_val_loss']:.6f}")
print(f"Best Validation Loss: {results['best_val_loss']:.6f}")

print("\nTest Metrics:")
for metric, value in results["test_metrics"].items():
    print(f"  {metric}: {value:.6f}")

Successfully loaded data from /home/lianlian/models/uni2ts/data/demand_data_all_cleaned_featsel.csv
Data shape: (61369, 8)
Columns: ['actual', 'met', 'period', 'hour', 'tempC', 'heatIndexC', 'feelsLikeC', 'humidity']


INFO:chronosx_train_eval:Starting training...
INFO:chronosx_train_eval:Starting training for 50 epochs
Training:   0%|          | 0/2265 [00:00<?, ?it/s]


TypeError: BaseException.with_traceback() takes exactly one argument (0 given)

In [ ]:
# Example hyperparameter search
param_grid = {
    "learning_rate": [1e-4, 5e-4, 1e-3, 5e-3],
    "batch_size": [8, 16, 32],
    "hidden_dim": [128, 256, 512],
}

hp_results = hyperparameter_search(
    base_model_config=model_config,
    base_data_config=data_config,
    param_grid=param_grid,
    n_trials=5,
)

print(f"\nBest hyperparameters: {hp_results['best_config']}")
print(f"Best validation loss: {hp_results['best_score']:.6f}")
